In [1]:
globals().clear(); import rwlib; import importlib,sunflow; importlib.reload(sunflow)
from sunflow import Location, Material, Product, Supplier, Producer,Customer, SupplyNet, Freight, InitSunflow, Capacity, Logistics, Demand,SunFlow,Component

sun = SunFlow()

mEngine  = Component('Engine')
mCarBody = Component('Car Body Parts')
mAccu    = Component('Accumulator')

p1 = Product('Porsche Taycan electric').component(mEngine,0.2).component(mCarBody,0.7).component(mAccu,0.1)
p2 = Product('Porsche Taycan electric').component(mEngine,0.1).component(mCarBody,0.8).component(mAccu,0.1).substituting(p1)
#
# !!! adapt to your data directory settings
#
sun.dataDir('py/pydata/datasets/').loadFreights('PorscheTaycan.xlsx')

Frankfurt    = sun.location('Frankfurt')
Chicago      = sun.location('Chicago')
Toronto      = sun.location('Toronto')
Zuffenhausen = sun.location('Zuffenhausen')
Shanghai     = sun.location('Shanghai')
Tokyo        = sun.location('Tokyo')
Seoul        = sun.location('Seoul')
Detroit      = sun.location('Detroit')
Stuttgart    = sun.location('Stuttgart')
Nanjing      = sun.location('Nanjing')
Chongqing    = sun.location('Chongqing')
Rotterdam    = sun.location('Rotterdam')
NewJersey    = sun.location('NewJersey')
Paris        = sun.location('Paris')

sElektromotoren  = Supplier('Elektromotoren GmbH')      .at(Stuttgart)
sMingHuang       = Supplier('Ming Huang Car Bodies Ltd').at(Nanjing)
sBatteriesCN     = Supplier('Batteries China Ltd')      .at(Chongqing)
sElectricEngines = Supplier('Electrical Engines Corp')  .at(Detroit)

plant_zuff       = Producer('Plant Zuffenhausen').at(Zuffenhausen).capacity(2500)  .varcost(20000)
plant_shang      = Producer('Plant Shanghai')    .at(Shanghai)    .capacity(800)   .varcost(10000)

wh_rotterdam     = Logistics('Distribution Center Rotterdam') .at(Rotterdam)
wh_newjersey     = Logistics('Distribution Center NewJersey') .at(NewJersey)
wh_shanghai      = Logistics('Distribution Center Shanghai')  .at(Shanghai)

frankfurt_market = Customer('Germany')    .at(Frankfurt)  .demand(300)
paris_market     = Customer('France')     .at(Paris)      .demand(200)
chicago_market   = Customer('USA')        .at(Chicago)    .demand(600)
toronto_market   = Customer('Canada')     .at(Toronto)    .demand(200)
china_market     = Customer('China')      .at(Shanghai)   .demand(1000)
japan_market     = Customer('Japan')      .at(Tokyo)      .demand(500)
korea_market     = Customer('Korea')      .at(Seoul)      .demand(400)


n = SupplyNet('Porsche Taycan electric').quantityUnit('pc').currencyUnit('$')

m_engine1 = mEngine .component(mEngine.name()) .by(sElektromotoren)   .varcost(15000)
m_engine2 = mEngine .component(mEngine.name()) .by(sElectricEngines)  .varcost(20000)
m_carbody = mCarBody.component(mCarBody.name()).by(sMingHuang)        .varcost(25000)
m_accu    = mAccu   .component(mAccu.name())   .by(sBatteriesCN)      .varcost(10000)


p_eu   = p1.assembly( m_engine1, m_carbody, m_accu, m_engine2) .at(plant_zuff) .fixcost(30000)
p_apac = p2.assembly( m_engine1, m_carbody, m_accu, m_engine2) .at(plant_shang).fixcost(25000)


p_market_eu    = n.distribution(p_eu,p_apac).by(wh_rotterdam)
p_market_apac  = n.distribution(p_eu,p_apac).by(wh_shanghai)
p_market_na    = n.distribution(p_eu,p_apac).by(wh_newjersey)


n.distribution(p_market_apac, p_market_eu, p_market_na).to(frankfurt_market) .title('Europe')
n.distribution(p_market_apac, p_market_eu, p_market_na).to(paris_market)     .title('Europe')
n.distribution(p_market_apac, p_market_eu, p_market_na).to(chicago_market)   .title('North America')
n.distribution(p_market_apac, p_market_eu, p_market_na).to(toronto_market)   .title('North America')
n.distribution(p_market_apac, p_market_eu)             .to(china_market)     .title('Asia Pacific')
n.distribution(p_market_apac, p_market_eu)             .to(japan_market)     .title('Asia Pacific')
n.distribution(p_market_apac, p_market_eu)             .to(korea_market)     .title('Asia Pacific')

n.compile()

n.showGraph().view()
opt = n.execute()
opt.showGraph().view()
opt.showGraph(flowOnly=True, orient='BT').view()


a=0


***Error: Product Porsche Taycan electric has unbalanced ingredient quantities sum = 0.9999999999999999 => should be 1.0 !
   Porsche Taycan electric
      Engine 0.2
      Car Body Parts 0.7
      Accumulator 0.1
